In [ ]:
import requests
from dotenv import load_dotenv
from constants import API_KEY
import dlt 
from pathlib import Path
import time 

load_dotenv()

db_path = str(Path(__file__).parents[1]/"data_warehouse/football_players.duckdb")

api_key = API_KEY
headers = {
    'x-apisports-key': api_key,
}
url = "https://v3.football.api-sports.io"


def _get_url(endpoints, params):
    url_for_search = f"{url}/{endpoints}"
    response = requests.get(url_for_search, headers= headers, params= params)
    response.raise_for_status()
    time.sleep(8)
    return response.json()


def get_team_id(league_id = 113, season= 2021):
    endpoints = "standings"
    
    params = {
        "league": league_id,
        "season": season,
    }
    
    data = _get_url(endpoints=endpoints, params= params)
    
    resp = data.get("response") or []
    if not resp:
        print("[DEBUG] standings: tom response"); return []
        
    team = resp[0]["league"]["standings"][0]
    all_id = [team_id["team"]["id"] for team_id in team] 
    
    return all_id 

@dlt.resource(write_disposition="replace")
def get_player(league_id = 113, season = 2021):
    
    teams_id = get_team_id(league_id= league_id, season= season)
    for team_id in teams_id:
        endpoints = "players"
        
        page = 1
        
        while True:
            params = {
                "team": team_id,
                "season": season,
                "page": page
            }
                
            data = _get_url(endpoints= endpoints, params= params)
            
            batch = data.get("response", [])
            
            if not batch:
                print(f"[DEBUG] Tom batch på sidor {page}. Result= {data.get('results')} errors= {data.get('errors')}")
                break 
            
            for player in batch:
                yield player 
            
                
            paging = data.get("paging") or {}
            if (paging.get("current") == paging.get("total")) or (paging.get("current") and paging.get("current") >=3):
                print(f"[DEBUG] Sista sidan ({page})")
                break 
            
            page += 1
                
    
            
def run_pipeline(table_name):
    pipeline = dlt.pipeline(
        pipeline_name = "swedish_fp",
        destination= dlt.destinations.duckdb(db_path),
        dataset_name= "staging",
    )
    
    load_info = pipeline.run(get_player(),table_name= table_name)
    print(load_info)
    
if __name__=="__main__":
    run_pipeline(table_name= "allsvenska_player")